In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.Chem import Draw
from collections import Counter
import matplotlib.pyplot as plt
import os

plt.rcParams['font.sans-serif'] = ['Arial', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("Reading CSV file...")
file_path = "/Users/bz999/Downloads/Stage II Round II.csv"
output_csv_path = file_path.replace(".csv", "_scaffold_stats.csv")

try:
    df = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
    exit()

if 'smiles' not in df.columns:
    print("Error: Column 'smiles' not found")
    print(f"Columns found: {df.columns.tolist()}")
else:
    smiles_list = df['smiles'].dropna().tolist()
    print(f"Successfully read {len(smiles_list)} SMILES strings")

print("Converting SMILES to molecule objects...")
mols = []
valid_smiles = []
for i, smiles in enumerate(smiles_list):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        mols.append(mol)
        valid_smiles.append(smiles)
    else:
        print(f"Warning: Failed to parse SMILES at index {i+1}: {smiles}")

print(f"Successfully converted {len(mols)} valid molecules")

print("Extracting Bemis-Murcko scaffolds...")
scaffold_smiles_list = []
scaffold_mols = []

for mol in mols:
    try:
        scaffold_mol = MurckoScaffold.GetScaffoldForMol(mol)
        scaffold_smiles = Chem.MolToSmiles(scaffold_mol)
        scaffold_smiles_list.append(scaffold_smiles)
        scaffold_mols.append(scaffold_mol)
    except Exception as e:
        print(f"Error extracting scaffold: {e}")
        continue

print("Counting scaffolds...")
scaffold_counts = Counter(scaffold_smiles_list)

print("Saving statistics to CSV...")
df_stats = pd.DataFrame(scaffold_counts.most_common(), columns=['Scaffold_SMILES', 'Count'])
df_stats.to_csv(output_csv_path, index=False)
print(f"Statistics saved successfully to: {output_csv_path}")

unique_scaffolds = len(scaffold_counts)
print(f"\n=== Analysis Results ===")
print(f"Total molecules processed: {len(mols)}")
print(f"Unique scaffolds found: {unique_scaffolds}")

print(f"\n=== Top 10 Scaffolds ===")
top_scaffolds = scaffold_counts.most_common(10)
top_scaffold_smiles = []
top_scaffold_mols = []
top_scaffold_counts = []

for i, (scaffold_smiles, count) in enumerate(top_scaffolds, 1):
    print(f"Rank {i}: Count={count}, SMILES={scaffold_smiles}")
    top_scaffold_smiles.append(scaffold_smiles)
    top_scaffold_mols.append(Chem.MolFromSmiles(scaffold_smiles))
    top_scaffold_counts.append(count)

print("Generating visualization...")
legends = [f"Rank {i}\nCount: {count}" for i, (count, smiles) in enumerate(zip(top_scaffold_counts, top_scaffold_smiles), 1)]

if top_scaffold_mols:
    img = Draw.MolsToGridImage(
        top_scaffold_mols,
        molsPerRow=4,
        subImgSize=(400, 400),
        legends=legends,
        returnPNG=False
    )
    
    plt.figure(figsize=(16, 12))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'Top 10 Bemis-Murcko Scaffolds\n(Total Unique Scaffolds: {unique_scaffolds})', fontsize=16, pad=20)
    plt.tight_layout()
    plt.show()
else:
    print("No valid scaffolds to visualize")

print(f"\n=== Scaffold Diversity Analysis ===")
total_scaffolds = len(scaffold_smiles_list)
if total_scaffolds > 0:
    diversity_ratio = unique_scaffolds / total_scaffolds
    print(f"Scaffold diversity ratio: {diversity_ratio:.3f} (closer to 1 means higher diversity)")

print("\nAnalysis complete!")